# Documentation for the creation and usage of the heatpump library (hplib)




In [1]:
import hplib as hpl

import database as db

## Preparation
1. we downloaded all manufacturer data from https://keymark.eu/en/products/heatpumps/certified-products
2. then we unzipped the files and used the bash-skript to convert pdf into txt
3. afterwards we used this following functions to create and extent the heatpump keymark database

In [5]:
# Import keymark data and save to csv databasedb.ImportKeymarkData()
# -> this creates /output/database_keymark.csv)

In [6]:
# Reduce to climate measurement series with average climate, delete redundant entries and save to csv sub-databasedb.ReduceKeymarkData('database_keymark.csv','average')
# -> this creates /output/database_keymark_average.csv)

In [7]:
# Normalize electrical and thermal power from the keymark database to values from setpoint T_in = -7 °C and T_out = 52 °Cdb.NormalizeKeymarkData('database_keymark_average.csv')
# -> this creates /output/database_keymark_average_normalized.csv)

In [8]:
# Identify subtypes like On-Off, Inverter and 2-stages Heatpumpdb.IdentifySubtypes('database_keymark_average_normalized.csv')
# -> this creates /output/database_keymark_average_normalized_subtypes.csv)

In [17]:
#Every heatpump gets a group depending on its type and subtypedb.AssignGroup('database_keymark_average_normalized_subtypes.csv')
# -> this overwrites /output/database_keymark_average_normalized_subtypes.csv)

In [2]:
#Calculate parameters p1-p9db.CalculateFunctionParameters('database_keymark_average_normalized_subtypes_subtypes.csv')
# -> this creates the final hplib-database.csv

KeyError: 'P_el_n_para_key'

In [25]:
Power=simulate.getPowerHP(5,50,simulate.getParaeasy('LW 101'))
Power=list(Power)
print('Thermal Power output: '+str(int((Power[0])))+' W')
print('Electrical Input for heatpump: '+str(int((Power[1])))+' W')
print('Electrical Input for Supplementary Heater: '+str(int((Power[2])))+' W')
print('COP: '+str(int((Power[3]))))

Thermal Power output: 10437 W
Electrical Input for heatpump: 3331 W
Electrical Input for Supplementary Heater: 0 W
COP: 3
